# Detailed Analysis (Bid by Bid, Card by Card)

This is a tutorial of how to do a detailed analysis of a played board.

The engine looks at the bidding and play as it originally happened, and does an analysis for every bid and every card played.

The analysis is not just a double-dummy analysis for the exact current layout (like if you would press the "GIB" button on BBO). Instead, it's an analysis over many different possible layouts (samples).

In [1]:
import os
os.chdir('..')

from nn.models import Models
from analysis import CardByCard
from util import parse_lin, display_lin
from sample import Sample
import conf

In [2]:
models = Models.from_conf(conf.load('./config/default_tf1x.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default_tf1x.conf'))  # Load sampling strategies
# For some strange reason PIMC is crashing the second time it is called from Jupyter
models.pimc_use_declaring = False
models.pimc_use_defending = False


Instructions for updating:
non-resource variables are not supported in the long term


INFO:tensorflow:Restoring parameters from ..\models/GIB/2024-07-06_bidding_V2-3114000
INFO:tensorflow:Restoring parameters from ..\models/GIB/2024-07-06_bidding_V2-3114000
INFO:tensorflow:Restoring parameters from ..\models/contract/contract-193200
INFO:tensorflow:Restoring parameters from ..\models/GIB/binfo_V2-1420000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/lr3_model/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\models/single_dummy/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\models/playing/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/playing/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/playing/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..

In [3]:
# we specify all the information about a board
# (it's quite tedious to enter every single thing by hand here,
# later we'll have an example of how you can give it a board played on BBO)

dealer = 'S'
vuln = [True, True]  # fist element is NS, second element is EW

hands = [
    'AJ87632.J96.753.',
    'K9.Q8542.T6.AJ74',
    'QT4.A.KJ94.KQ986',
    '5.KT73.AQ82.T532'
]

auction = ['1N', 'PASS', '4H', 'PASS', '4S', 'PASS', 'PASS', 'PASS']

play = ['C2', 'D3', 'CA', 'C6', 'D6', 'DJ', 'DQ', 'D5', 'DA', 'D7', 'DT', 'D4', 'D8', 'H6', 'H2', 'D9', 'SQ', 'S5', 'S2', 'SK', 'H4', 'HA', 'H7', 'H9', 'S4', 'C3', 'SA', 'S9', 'S3', 'C4', 'ST', 'H3', 'CK', 'C5', 'HJ', 'C7', 'C8', 'CT', 'S6', 'CJ', 'S7', 'H8', 'C9', 'D2', 'S8', 'H5', 'CQ', 'HT', 'SJ', 'HQ', 'DK', 'HK']

In [4]:
card_by_card = CardByCard(dealer, vuln, hands, auction, play, models, sampler, False)

Loaded lib dds.dll
DDSolver being loaded - dds mode 1


In [5]:
# calling this starts the analysis
# it will go bid-by-bid and card-by-card, and will take a few moments
# possible mistakes will be annotated with ? or with ?? (if it's a bigger mistake)
# (possible mistake means that the engine does not agree with the bid/play. the engine could be wrong too :))

await card_by_card.analyze()

analyzing the bidding


DDSolver being loaded - dds mode 1


1N Suggested bid from NN: CandidateBid(bid=1C  , insta_score=0.9999, expected_score=---, expected_mp=---, expected_imp=---, expected_tricks=---, adjust=---, alert=  )
1N is not in the bids from the neural network
PASS OK NN-value: 1.000
4H OK NN-value: 0.993
PASS OK NN-value: 0.981
4S OK NN-value: 1.000
PASS OK NN-value: 1.000
PASS OK NN-value: 0.998
PASS OK NN-value: 1.000
analyzing opening lead
C2
C2 OK
analyzing play


DDSolver being loaded - dds mode 1


D3 ? losing: 0.53
CA ?? losing: 0.65
C6 OK
D6 OK
DJ OK
DQ Forced
D5 OK
DA OK
D7 OK
DT OK
D4 OK
D8 OK
H6 OK
H2 OK
D9 OK
SQ OK
S5 OK
S2 OK
SK OK
H4 OK
HA OK
H7 OK
H9 OK
S4 OK
C3 Forced
SA ?? losing: 1.14
S9 Forced
S3 OK
C4 OK
ST OK
H3 OK
CK OK
C5 OK
HJ Forced
C7 OK
C8 OK
CT Forced
S6 OK
CJ OK
S7 OK
H8 OK
C9 Forced
D2 OK
S8 OK
H5 Forced
CQ OK
HT OK


In [6]:
# the engine does not agree with the 1N opening.
# indeed, it's a little offbeat with a singleton
# let's see what the engine is thinking (what would it bid instead)

card_by_card.bid_responses[0].to_dict()  # the 0 index is the first bid in the auction

{'bid': '1N',
 'who': 'Analysis',
 'quality': '1.0',
 'candidates': [{'call': '1C', 'insta_score': 1.0, 'alert': 'False'}],
 'samples': ['K8x.Q9x.QTx.JTxx J9x.Txxxx.xx.Axx QTx.A.KJ9x.KQ98x Axxx.KJ8x.A8xx.x 1.00000',
  'Ax.QJT9xx.Tx.Jxx K9xxx.8x.Qxxx.Tx QTx.A.KJ9x.KQ98x J8x.Kxxx.A8x.Axx 1.00000',
  'A8x.KQJTx.A8.ATx 9x.9xxx.Txxx.xxx QTx.A.KJ9x.KQ98x KJxxx.8xx.Qxx.Jx 1.00000',
  'K9xx.xxx.QTx.ATx xxx.K9x.Axx.Jxxx QTx.A.KJ9x.KQ98x AJ8.QJT8xx.8xx.x 1.00000',
  'K8xx.T9x.Qxxx.Tx Axx.KJxxxx.xx.Jx QTx.A.KJ9x.KQ98x J9x.Q8x.AT8.Axxx 1.00000',
  'xxxx.Qxx.AQxx.Jx KJ98x.T9xx.8x.xx QTx.A.KJ9x.KQ98x A.KJ8xx.Txx.ATxx 1.00000',
  'K8xx.QJxx.Qx.xxx AJxxx.Txx.T8xx.A QTx.A.KJ9x.KQ98x 9.K98xx.Axx.JTxx 1.00000',
  '8xx.QTxx.Q8xxx.x AKx.K8xx.xx.Axxx QTx.A.KJ9x.KQ98x J9xx.J9xx.AT.JTx 1.00000',
  'AKxxx.KT.xx.Txxx J9x.98xx.AQ8x.Ax QTx.A.KJ9x.KQ98x 8x.QJxxxx.Txx.Jx 1.00000',
  'A8xx.J9x.Qxx.Jxx J9xx.Kxx.8xx.ATx QTx.A.KJ9x.KQ98x Kx.QT8xxx.ATx.xx 1.00000',
  'x.KJ8xxx.Qxx.Jxx KJ98x.Q9.ATxx.Tx QTx.A.KJ9x.KQ98x A

the engine very confidently opens `1C` and doesn't even consider `1N`

In [7]:
# what about the opening lead? let's see...

card_by_card.cards['C2'].to_dict()

{'card': 'C2',
 'who': '',
 'quality': '0.908',
 'hcp': [6.1, 2.4, 2.1, 2.3, 2.6, 3.2, 3.6, 3.5, 2.9, 3.4, 3.3, 3.3],
 'shape': [8.3, 7.1, 15.7],
 'candidates': [{'card': 'Sx',
   'insta_score': 0.474,
   'expected_tricks_dd': 10.22,
   'p_make_contract': 0.28,
   'expected_score_dd': 431},
  {'card': 'Cx',
   'insta_score': 0.307,
   'expected_tricks_dd': 10.21,
   'p_make_contract': 0.28,
   'expected_score_dd': 428},
  {'card': 'Hx',
   'insta_score': 0.148,
   'expected_tricks_dd': 10.35,
   'p_make_contract': 0.28,
   'expected_score_dd': 428},
  {'card': 'DA',
   'insta_score': 0.054,
   'expected_tricks_dd': 10.41,
   'p_make_contract': 0.2,
   'expected_score_dd': 494}],
 'samples': ['x.KTxx.AQ8x.Txxx AT98xx.xx.xxx.Ax KJx.J98xx.9x.Q9x Qxx.AQ.KJTx.KJ8x 0.94189',
  'x.KTxx.AQ8x.Txxx JT98xxxx.xx.9x.Q K.A98.JTxx.A9xxx AQx.QJxx.Kxx.KJ8 0.99854',
  'x.KTxx.AQ8x.Txxx J98xxx.A8.KJ.Q8x KTxx.9xxx.xxx.Jx AQ.QJx.T9xx.AK9x 0.78662',
  'x.KTxx.AQ8x.Txxx AQ9xxxx.x.xx.KQ9 8x.J9x.JT9x.J8xx KJT.

the engine agrees with leading a low club, but it's very close. the alternative is a low heart

In [8]:
# the engine considers dummy's discard of D3 on the first trick a big mistake.
# perhaps we should ruff instead, let's see what the engine suggests

card_by_card.cards['D3'].to_dict()

{'card': 'S2',
 'who': 'NN-Make',
 'quality': '0.8506',
 'hcp': [8.8, 9.3],
 'shape': [2.0, 3.3, 3.2, 4.5, 2.1, 3.2, 3.2, 4.4],
 'candidates': [{'card': 'S6',
   'insta_score': 0.613,
   'expected_tricks_dd': 11.11,
   'p_make_contract': 0.96,
   'expected_score_dd': 625,
   'msg': ''},
  {'card': 'S2',
   'insta_score': 0.613,
   'expected_tricks_dd': 11.11,
   'p_make_contract': 0.96,
   'expected_score_dd': 625,
   'msg': ''},
  {'card': 'S3',
   'insta_score': 0.613,
   'expected_tricks_dd': 11.11,
   'p_make_contract': 0.96,
   'expected_score_dd': 625,
   'msg': ''},
  {'card': 'S7',
   'insta_score': 0.613,
   'expected_tricks_dd': 11.11,
   'p_make_contract': 0.96,
   'expected_score_dd': 625,
   'msg': ''},
  {'card': 'S8',
   'insta_score': 0.034,
   'expected_tricks_dd': 11.11,
   'p_make_contract': 0.96,
   'expected_score_dd': 625,
   'msg': ''},
  {'card': 'SJ',
   'insta_score': 0.011,
   'expected_tricks_dd': 11.02,
   'p_make_contract': 0.95,
   'expected_score_dd': 61

indeed, the best play is to ruff low.

looking at the samples, we see that East has the `CA` in every sample (this is by inference because underleading an A is very unlikely)

## Analyzing a board played on BBO

In [9]:
# copy-paste from the hand records (in lin format)

lin = 'pn|You,~~M7228oka,~~M72302cm,~~M72316sq|st||md|1S4TKHJD68QC679TKA,S35H479TQKD24TAC8,S2789H3AD379JKC35,|rh||ah|Board 3|sv|e|mb|1C|an|Minor suit opening -- 3+ !C; 11-21 HCP; 12-22 total points|mb|2H|an|Aggressive weak jump overcall -- 6+ !H; 4-10 HCP |mb|d|an|Negative double -- 4+ !S; 7+ HCP; 8+ total points |mb|4H|an|The Law: 10 trump -> game support -- 4+ total points |mb|4S|an|3+ !C; 4+ !S; 16-21 HCP; 17-22 total points|mb|p|mb|p|mb|p|pg||pc|DA|pc|D3|pc|D5|pc|D6|pg||pc|C8|pc|C3|pc|CJ|pc|CA|pg||pc|S4|pc|S5|pc|S8|pc|SJ|pg||pc|H5|pc|HJ|pc|HQ|pc|HA|pg||pc|S2|pc|SA|pc|ST|pc|S3|pg||pc|H2|pc|SK|pc|H4|pc|H3|pg||pc|D8|pc|D2|pc|DJ|pc|S6|pg||pc|SQ|pc|C6|pc|H7|pc|S7|pg||pc|H8|pc|C7|pc|HK|pc|S9|pg||pc|C5|pc|C2|pc|CT|pc|HT|pg||pc|CK|pc|H9|pc|D7|pc|C4|pg||pc|DQ|pc|D4|pc|DK|pc|H6|pg||pc|D9|pc|CQ|pc|C9|pc|DT|pg||'

In [10]:
display_lin(lin)

In [11]:
board = parse_lin(lin)
print(board)

Board(dealer='S', vuln=[False, True], hands=['9872.A3.KJ973.53', 'JQ6A.5826.5.2JQ4', 'KT4.J.Q86.AKT976', '53.KQT974.AT42.8'], auction=['1C', '2H', 'X', '4H', '4S', 'PASS', 'PASS', 'PASS'], play=['DA', 'D3', 'D5', 'D6', 'C8', 'C3', 'CJ', 'CA', 'S4', 'S5', 'S8', 'SJ', 'H5', 'HJ', 'HQ', 'HA', 'S2', 'SA', 'ST', 'S3', 'H2', 'SK', 'H4', 'H3', 'D8', 'D2', 'DJ', 'S6', 'SQ', 'C6', 'H7', 'S7', 'H8', 'C7', 'HK', 'S9', 'C5', 'C2', 'CT', 'HT', 'CK', 'H9', 'D7', 'C4', 'DQ', 'D4', 'DK', 'H6', 'D9', 'CQ', 'C9', 'DT'])


In [12]:
card_by_card = CardByCard(*board, models, sampler, False)


DDSolver being loaded - dds mode 1


In [13]:
await card_by_card.analyze()

analyzing the bidding


DDSolver being loaded - dds mode 1


1C OK NN-value: 1.000
2H OK NN-value: 0.998
X Suggested bid from NN: CandidateBid(bid=PASS, insta_score=0.9954, expected_score=---, expected_mp=---, expected_imp=---, expected_tricks=---, adjust=---, alert=  )
X is not in the bids from the neural network
4H Suggested bid from NN: CandidateBid(bid=2S  , insta_score=0.3332, expected_score=   27, expected_mp=---, expected_imp=---, expected_tricks= 8.84, adjust=16.66, alert=  )
4H is not in the bids from the neural network
4S Suggested bid from NN: CandidateBid(bid=PASS, insta_score=0.9568, expected_score=---, expected_mp=---, expected_imp=---, expected_tricks=---, adjust=---, alert=  )
4S is not in the bids from the neural network
PASS OK NN-value: 0.998
PASS OK NN-value: 0.999
PASS OK NN-value: 0.757
analyzing opening lead
DA
DA OK
analyzing play


DDSolver being loaded - dds mode 1


D3 OK
D5 Forced
D6 OK
C8 OK
C3 OK
CJ OK
CA OK
S4 ? losing: 0.55
S5 OK
S8 OK
SJ ?? losing: 0.74
H5 ? losing: 0.13
HJ OK
HQ ?? losing: 1.92
HA Forced
S2 OK
SA OK
ST OK
S3 Forced
H2 ? losing: 0.34
SK OK
H4 OK
H3 ?? losing: 1.00
D8 OK
D2 OK
DJ OK
S6 OK
SQ OK
C6 OK
H7 Follow
S7 Forced
H8 OK
C7 Forced
HK OK
S9 OK
C5 OK
C2 OK
CT OK
HT OK
CK OK
H9 OK
D7 OK
C4 OK
DQ OK
D4 OK
DK OK
H6 Forced


the engine agrees with the bidding, but didn't like something in the cardplay.

playing `S4` from hand is the first mistake. apparently this play drops almost half a trick on average.

In [14]:
card_by_card.cards['S4'].to_dict()

{'card': 'DQ',
 'who': 'NN-Make',
 'quality': '0.1661',
 'hcp': [6.7, 8.3],
 'shape': [3.4, 3.8, 3.3, 2.4, 2.2, 6.0, 2.7, 2.0],
 'candidates': [{'card': 'DQ',
   'insta_score': 0.095,
   'expected_tricks_dd': 7.04,
   'p_make_contract': 0.0,
   'expected_score_dd': -98,
   'msg': ''},
  {'card': 'D8',
   'insta_score': 0.046,
   'expected_tricks_dd': 7.04,
   'p_make_contract': 0.0,
   'expected_score_dd': -98,
   'msg': ''},
  {'card': 'HJ',
   'insta_score': 0.414,
   'expected_tricks_dd': 7.03,
   'p_make_contract': 0.0,
   'expected_score_dd': -98,
   'msg': ''},
  {'card': 'CK',
   'insta_score': 0.168,
   'expected_tricks_dd': 6.9,
   'p_make_contract': 0.0,
   'expected_score_dd': -105,
   'msg': ''},
  {'card': 'S4',
   'insta_score': 0.192,
   'expected_tricks_dd': 6.49,
   'p_make_contract': 0.0,
   'expected_score_dd': -128,
   'msg': 'trump adjust=0.05'},
  {'card': 'ST',
   'insta_score': 0.057,
   'expected_tricks_dd': 6.49,
   'p_make_contract': 0.0,
   'expected_score_d

the opening lead of `DA` is interesting. the engine prefers the `HK` and it's the only card it considers.

In [15]:
card_by_card.cards['DA'].to_dict()

{'card': 'DA',
 'who': '',
 'quality': '0.782',
 'hcp': [4.3, 1.7, 3.6, 3.4, 2.6, 4.1, 3.0, 3.2, 4.2, 1.4, 2.3, 5.3],
 'shape': [11.4, 5.6, 13.7],
 'candidates': [{'card': 'DA',
   'insta_score': 0.057,
   'expected_tricks_dd': 11.43,
   'p_make_contract': 0.01,
   'expected_score_dd': 458},
  {'card': 'C8',
   'insta_score': 0.428,
   'expected_tricks_dd': 11.73,
   'p_make_contract': 0.01,
   'expected_score_dd': 468},
  {'card': 'HK',
   'insta_score': 0.44,
   'expected_tricks_dd': 11.67,
   'p_make_contract': 0.01,
   'expected_score_dd': 468},
  {'card': 'Sx',
   'insta_score': 0.065,
   'expected_tricks_dd': 11.79,
   'p_make_contract': 0.01,
   'expected_score_dd': 472}],
 'samples': ['xx.KQT9xx.ATxx.8 KJT9.Ax.Kxx.Q9xx xx.J8xxx.J98x.Jx AQ8xx..Qx.AKTxxx 0.76074',
  'xx.KQT9xx.ATxx.8 QT8x.A8.QJ8.AJxx 9xx.Jxxx.9xx.T9x AKJx.x.Kxx.KQxxx 0.76709',
  'xx.KQT9xx.ATxx.8 QTxx.Ax.KJ8.KQxx 8x.J8xxx.Qxxx.9x AKJ9x..9x.AJTxxx 0.87695',
  'xx.KQT9xx.ATxx.8 QT9x.8.KQ98x.Axx 8xx.Jxxxx.xx.KTx AKJ